In [4]:
import pandas as pd
import pandas_profiling
import numpy as np

In [5]:
%%javascript
require.config({
paths: {
d3: "https://d3js.org/d3.v5.min",
}
});

require(["d3"], function(d3) {
window.d3 = d3;
});

<IPython.core.display.Javascript object>

In [ ]:
columns = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height",
           "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]

df = pd.read_csv('imports-85.data', names=columns)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

pandas_profiling.ProfileReport(df)

In [ ]:
columns = ["class", "age", "menopause", "tumor-size", "inv-nodes", 
           "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]

df = pd.read_csv('breast-cancer.data', names=columns)

df['deg-malig'].replace(to_replace={1: 'one', 2:'two', 3:'three'}, inplace=True)

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)


pandas_profiling.ProfileReport(df)

In [24]:
columns = ["class","age","sex","histologic-type","degree-of-diffe","bone","bone-marrow","lung","pleura","peritoneum","liver","brain","skin","neck","supraclavicular","axillar","mediastinum","abdominal"]

df = pd.read_csv('primary-tumor.data', names=columns)

df.replace(to_replace=["na", "?", np.nan, "missing", "not available","n/a", "missing value"], value=np.nan, inplace=True)

for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='ignore')

df["class"].replace(to_replace= {1:"lung", 2:"head & neck", 3:"esophasus", 4:"thyroid", 5:"stomach", 6:"duoden & sm.int", 7:"colon", 8:"rectum", 9:"anus", 10:"salivary glands", 11:"pancreas", 12:"gallblader", 13:"liver", 14:"kidney", 15:"bladder", 16:"testis", 17:"prostate", 18:"ovary", 19:"corpus uteri", 20: "cervix uteri", 21:"vagina", 22:"breast"}, inplace=True)

df["age"].replace(to_replace ={1:"<30",2:"30-59",3:">60"}, inplace=True)
df["sex"].replace(to_replace ={1:"male",2:"female"}, inplace=True)
df["histologic-type"].replace(to_replace ={1.0:"epidermoid",2.0:"adeno", 3.0:"anaplastic"}, inplace=True)

df


,class,age,sex,histologic-type,degree-of-diffe,bone,bone-marrow,lung,pleura,peritoneum,liver,brain,skin,neck,supraclavicular,axillar,mediastinum,abdominal
0,lung,<30,male,NaN,3.0,2,2,1,2,2,2,2,2.0,2,2,2.0,2,2
1,lung,<30,male,NaN,3.0,2,2,2,2,2,1,2,2.0,2,1,2.0,1,2
2,lung,<30,female,adeno,3.0,1,2,2,2,2,2,2,2.0,2,2,2.0,1,2
3,lung,<30,female,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
4,lung,<30,female,NaN,3.0,1,2,1,1,2,2,2,2.0,2,2,2.0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,breast,30-59,female,adeno,NaN,2,2,2,2,2,2,2,2.0,2,2,1.0,2,2
335,breast,30-59,female,adeno,NaN,2,2,2,2,2,2,2,2.0,2,2,1.0,2,2
336,breast,30-59,female,NaN,NaN,1,2,2,2,2,2,2,2.0,2,1,1.0,2,2
337,breast,>60,female,adeno,2.0,2,2,2,2,2,2,2,2.0,1,1,1.0,2,2
